In [373]:
from pymongo import MongoClient
import json
from bson.objectid import ObjectId
import math
import pandas as pd

def CalculoAngulo(ang_11,ang_22):
    """
        Calculamos ambos ángulo formado entre las dos antenas y el tag
    """
    ang_1 = ang_11
    ang_2 = ang_22

    if ang_1 < 0.0:
        if ang_2 < 0.0:
            Alfa = 90.0 + ang_1
            Beta = 90.0 - ang_2
            return Alfa,Beta
        #Caso 2:
        elif ang_2 > 0.0:
            Alfa = 90.0 + ang_1
            Beta = 90.0 - ang_2
            return Alfa,Beta
        #Caso 3:    
        elif ang_2 == 0.0:
            Alfa = 90.0 + ang_1
            Beta = 90.0
            return Alfa,Beta
    #caso 4:
    elif ang_1 > 0.0:
        if ang_2 > 0.0:
            Alfa = 90.0 + ang_1
            Beta = 90.0 - ang_2
            return Alfa,Beta
    #Caso 5:
    elif ang_1 == 0.0:
        if ang_2 > 0.0:
            Alfa = 90.0
            Beta = 90.0 + ang_2
            return Alfa,Beta

def Distancia_ant_tag(d_entre_ants,Alfa_1,Beta_1):
    """
    Distancia entre el tag y la antena
    - Distancia entre ambas antenas
    - Alfa corresponde al ángulo de la antena 1 
    - Beta corresponde al ángulo de la antena 2
    a la salida tendremos dos distancias 
    """
    alfa,beta = CalculoAngulo(Alfa_1,Beta_1)

    # Distancia desde el punto C a A
    # con a definido como la distancia entre las dos antenas
    sigma = 180-beta-alfa
    if(sigma!=0):
        B = abs(d_entre_ants * math.sin(math.radians(alfa))/math.sin(math.radians(sigma))) # b=a*sin(beta)/sin(sigma)
        A = abs(d_entre_ants * math.sin(math.radians(beta))/math.sin(math.radians(sigma))) # c=a*sin(alfa)/sin(sigma)
    else:
        A = 0
        B = 0
    # la altura del triangulo que se forma entre las dos distancias B y C
    #h = d_entre_ants * (math.sin(math.radians(alfa))*math.sin(math.radians(beta))) / math.sin(math.radians(alfa+beta))
    return B,A

def obtener_bd(direction="localhost", basedatos='memory_test',coleccion='data'):
    host = direction
    port = "27017"
    cliente = MongoClient("mongodb://{}:{}".format(host, port))
    db = cliente[basedatos]
    collection = db[coleccion]
    return collection

In [374]:

## Leemos el json con los datos de las antenas y el tag
f = open("Ant_tag.json", "r")
c = f.read()
f.close()
js = json.loads(c)

## Establecemos la conexion con el servidor
base_de_datos = obtener_bd()

## Traemos el ultimo elemento de la lista en el tiempo de lectura
## La idea es poder trabajar con el tiempo anterior a este número
r_0 = base_de_datos.find().sort("_id",-1).limit(1)

for r in r_0:
    id_0 = r

respuestas = base_de_datos.find().sort("_id",-1)

# tag_1/Ant_1
vect_11 = []
# tag_1/Ant_2
vect_12 = []

for r in respuestas:
    if(r['Id_tag']== js["tag_1"] and r['Id_Ant']==js["Antena_1"]):
        vect_11.append(r)
    elif(r['Id_tag']== js["tag_1"] and r['Id_Ant']==js["Antena_2"]):
        vect_12.append(r)
    else:
        pass
## De los cuatro vector que creamos, los reducimos a dos para poder tener las variables que tienen el mismo tag con antenas diferentes
## El críterio para juntar alguno de los elementos es que tengan la misma hora de captura del dato y que el tag sea igual en ambos casos pero los datos deben ser tomado de diferentes antenas
## En este caso tenemos solo dos antenas y dos tag

vect_1 = []
vect_2 = []
# cargamos los datos al vect_1
for r in vect_11:
    for s in vect_12:
        if(r['hora']==s['hora']):
            vect_1.append(r)
            vect_1.append(s)
            vect_12.remove(s)

In [375]:
vect_1_1 = pd.DataFrame(vect_1)
vect_1_1.info()

In [377]:
## Para ingresar a los elementos del arreglo y modificarlos
## Actualizo los datos añadiendo el parametro Distancia_ant_tag para guardarlos en la base de datos de MongoDb
## Vector 1
for t in range(int(len(vect_1)/2)):
    try:
        D_B,D_A = Distancia_ant_tag(vect_1[2*t]['Distancia_entre_ant'],vect_1[2*t]['Ang_azimuth'],vect_1[(2*t)+1]['Ang_azimuth'])
        d_a = D_A
        d_b = D_B
        base_de_datos.update_one(
            {"_id": ObjectId(str(vect_1[2*t]['_id']))},
            {
                "$set":{
                    "Distancia_ant_tag": int(d_b)
                }
            }
        )
        base_de_datos.update_one(
            {"_id": ObjectId(vect_1[(2*t)+1]['_id'])},
            {
                "$set":{
                    "Distancia_ant_tag": int(d_a)
                }
            }
        )
    except:
        #print(vect_1[2*t])
        #print(vect_1[(2*t)+1])
        pass
    

{'_id': ObjectId('639298a62c578993a2dc8d44'), 'hora': '2022/12/06/ 00:59:08', 'Id_tag': ':CCF957966AC9', 'Id_Ant': 'PID=0403:6015 SER=D200C017A', 'RSSI': -65, 'Ang_azimuth': 18, 'Ang_elevacion': -44, 'Canal': 39, 'Altura_ant(cm)': 170, 'Distancia_entre_ant_tag(cm)': 122, 'Error_dato_medido': 1, 'Distancia_entre_ant': 160}
{'_id': ObjectId('639298a62c578993a2dc93e6'), 'hora': '2022/12/06/ 00:59:08', 'Id_tag': ':CCF957966AC9', 'Id_Ant': 'PID=0403:6015 SER=D200BZVHA', 'RSSI': -65, 'Ang_azimuth': -30, 'Ang_elevacion': -34, 'Canal': 38, 'Altura_ant(cm)': 170, 'Distancia_entre_ant_tag(cm)': 216, 'Error_dato_medido': 1, 'Distancia_entre_ant': 160}
{'_id': ObjectId('639298a62c578993a2dc8d44'), 'hora': '2022/12/06/ 00:59:08', 'Id_tag': ':CCF957966AC9', 'Id_Ant': 'PID=0403:6015 SER=D200C017A', 'RSSI': -65, 'Ang_azimuth': 18, 'Ang_elevacion': -44, 'Canal': 39, 'Altura_ant(cm)': 170, 'Distancia_entre_ant_tag(cm)': 122, 'Error_dato_medido': 1, 'Distancia_entre_ant': 160}
{'_id': ObjectId('639298a62

Exporto los datos que estan listos

In [378]:
## Traemos el ultimo elemento de la lista en el tiempo de lectura
## La idea es poder trabajar con el tiempo anterior a este número
r_0 = base_de_datos.find().sort("_id",-1).limit(1)

for r in r_0:
    id_0 = r

respuestas = base_de_datos.find().sort("_id",-1)

# tag_1/Ant_1
vect_11 = []
# tag_1/Ant_2
vect_12 = []

for r in respuestas:
    if(r['Id_tag']== js["tag_1"] and r['Id_Ant']==js["Antena_1"]):
        vect_11.append(r)
    elif(r['Id_tag']== js["tag_1"] and r['Id_Ant']==js["Antena_2"]):
        vect_12.append(r)
    else:
        pass
## De los cuatro vector que creamos, los reducimos a dos para poder tener las variables que tienen el mismo tag con antenas diferentes
## El críterio para juntar alguno de los elementos es que tengan la misma hora de captura del dato y que el tag sea igual en ambos casos pero los datos deben ser tomado de diferentes antenas
## En este caso tenemos solo dos antenas y dos tag

vect_1 = []
vect_2 = []
# cargamos los datos al vect_1
for r in vect_11:
    for s in vect_12:
        if(r['hora']==s['hora']):
            vect_1.append(r)
            vect_1.append(s)
            vect_12.remove(s)

In [379]:
vect_1_1 = pd.DataFrame(vect_1)
vect_1_1.info()

In [381]:
vect_1_1.to_csv('Entrenamiento3\\Entrenamiento_242.csv',encoding = 'utf-8')